In [2]:
# Импорт библиотек
import numpy
import pandas as pd

# Импорт matplotlib - не нужен, на всякий случай

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline



Работа с пропущенными значениями

In [3]:
# Задаем рабочую папку

import os
os.chdir("C:/Users/USER/Documents/Python/__AD_Python_part_2_2019/00_data_transform")


In [3]:
# Импорт данных
# wine = pd.read_csv('Wine.txt', sep='\t', header=0)
df_01 = pd.read_csv('data_01.csv', sep=',', header=0)

In [4]:
df_01

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


Внимание!

При импорте данных с помощью pandas пропущенные значения 
заменяются на NaN (not a number)

В пакете R NA сокращение для Not Available)

Задача.  Сравнить каждый элемент таблицы с NaN

isnull method

In [5]:
df_01.isnull()

,A,B,C,D
0,False,False,False,False
1,False,False,True,False
2,False,False,False,True


Задача.  Сколько пропусков в каждом столбце

In [6]:
df_01.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

Задача.  Преобразовать Pandas DataFrame в NumPy array

Для работы с scikit-learn и/или TensorFlow необходимы NumPy arrays.

NaN станут немного другими

In [7]:
df_02 = df_01.values
print(df_02)

[[ 1.  2.  3.  4.]
 [ 5.  6. nan  8.]
 [10. 11. 12. nan]]


Задача.  Удалить строки (или столбцы), содержащие пропуски

dropna method

In [8]:
# Подзадача 1.   Удалить строки, содержащие хотя бы один NaN (пропуск)

df_01.dropna(axis=0)


,A,B,C,D
0,1.0,2.0,3.0,4.0


In [9]:
# Подзадача 2.   Удалить столбцы, содержащие хотя бы один NaN (пропуск)

df_01.dropna(axis=1)


,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [10]:
# Подзадача 3.   Удалить столбцы, содержащие только NaN

df_01.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [11]:
# Подзадача 4.   Удалить строки, содержащие менее 4 не NaN

df_01.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [12]:
# Подзадача 4.   Удалить строки, в которых NaN содержатся в определенных колонках
# В примере это столбец  'C'

df_01.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


Осторожно!
Можно удалить слишком много строк
Можно удалить информативные столбцы

Задача.  Заполнение пропущенных значений оценками

Неправильно: заменить пропуски на нули

In [13]:
# Подзадача 1.   Заполнение пропущенных значений средними по столбцам

# Imputer class from scikit-learn

from sklearn.preprocessing import Imputer
imr = Imputer(missing_values='NaN', strategy='mean', axis=0)
imr = imr.fit(df_02)
imputed_data = imr.transform(df_02)
imputed_data

# Внимание:  используем NumPy array

# Общий подход в scikit-learn
# Сначала fit (вычисляем оценки), потом transform (преобразуем, используя оценки из fit)

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

Подзадача 2.   Заполнение пропущенных значений средними по строкам 

Как выше, но вместо axis=0 используйте axis=1

Применяется очень редко!

Подзадача 3.   Заполнение пропущенных значений медианами (или модами) по столбцам

Как выше, но вместо strategy='mean' используйте strategy='median' (или strategy='most_frequent')



Работа с переменными в порядковой и номинальной шкалах

In [25]:
# Импорт данных
# wine = pd.read_csv('Wine.txt', sep='\t', header=0)
df_03 = pd.read_csv('data_02.csv', sep=',', header=0)

In [26]:
print(df_03)

     color   size  price classlabel
0  'green'    'M'   10.1   'class1'
1    'red'    'L'   13.5   'class2'
2   'blue'   'XL'   15.3   'class1'


Таблица df_03 содержит переменные в номинальной шкале ( color ), 
порядковой шкале ( size ), количественной шкале ( price )

In [27]:
df_03.columns

Index(['color', 'size', 'price', 'classlabel'], dtype='object')

In [28]:
df_03['size'] 

0      'M'
1      'L'
2     'XL'
Name: size, dtype: object

Задача. Заменить значения переменной, измеренной в порядковой шкале, на целые числа.

Порядок кодов важен. size_mapping - таблица перекодировки

Таблицу перекодировок придется составлять вручную!


In [29]:
#  Первая попытка неудачная, почему?

size_mapping = { 'XL': 3, 'L': 2, 'M': 1}

df_03["size_2"] = df_03["size"].map(size_mapping)

print(df_03)


     color   size  price classlabel  size_2
0  'green'    'M'   10.1   'class1'     NaN
1    'red'    'L'   13.5   'class2'     NaN
2   'blue'   'XL'   15.3   'class1'     NaN


In [31]:
#  Проблема в команде map или в данных?
#  Попробуем перекодировать элементы из другого столбца

size_mapping = { 10.1 : 3, 13.5 : 2, 15.3: 1}

df_03["size_2"] = df_03["price"].map(size_mapping)

print(df_03)


     color   size  price classlabel  size_2
0  'green'    'M'   10.1   'class1'       3
1    'red'    'L'   13.5   'class2'       2
2   'blue'   'XL'   15.3   'class1'       1


In [32]:
df_03['size']

0      'M'
1      'L'
2     'XL'
Name: size, dtype: object

In [33]:
df_03['size'][2]

" 'XL'"

In [35]:
size_mapping = { " 'XL'": 3, " 'L'": 2, " 'M'": 1}

df_03["size_2"] = df_03["size"].map(size_mapping)

print(df_03)


     color   size  price classlabel  size_2
0  'green'    'M'   10.1   'class1'       1
1    'red'    'L'   13.5   'class2'       2
2   'blue'   'XL'   15.3   'class1'       3


In [39]:
#  Обратное преобразование, возвращаемся к исходным кодам

inv_size_mapping = {v: k for k, v in size_mapping.items()} 


In [40]:
size_mapping.items()

dict_items([('XL', 3), ('L', 2), ('M', 1)])

In [41]:
df_03['size_3'] = df_03['size_2'].map(inv_size_mapping)

In [42]:
df_03

,color,size,price,classlabel,size_2,size_3
0,'green','M',10.1,'class1',1,M
1,'red','L',13.5,'class2',2,L
2,'blue','XL',15.3,'class1',3,XL


In [36]:
# Импорт данных
# wine = pd.read_csv('Wine.txt', sep='\t', header=0)
df_04 = pd.read_csv('data_03.csv', sep=',', header=0)

In [37]:
print(df_04)

   color size  price classlabel
0  green    M   10.1     class1
1    red    L   13.5     class2
2   blue   XL   15.3     class1


In [38]:
size_mapping = { "XL": 3, "L": 2, "M": 1}

df_04["size_2"] = df_04["size"].map(size_mapping)

print(df_04)


   color size  price classlabel  size_2
0  green    M   10.1     class1     NaN
1    red    L   13.5     class2     NaN
2   blue   XL   15.3     class1     NaN


Убедимся, что до сих пор мы неправильно представляли себе данные

In [23]:
df_04['size']

0      M
1      L
2     XL
Name: size, dtype: object

In [24]:
df_04['size'][2]

' XL'

In [43]:
size_mapping = { " XL": 3, " L": 2, " M": 1}

df_04["size_2"] = df_04["size"].map(size_mapping)

print(df_04)


   color size  price classlabel  size_2
0  green    M   10.1     class1       1
1    red    L   13.5     class2       2
2   blue   XL   15.3     class1       3


In [ ]:
Если переменная измерена в количественной шкале, порядок кодов не важен.

Новые коды могут быть произвольными числами

class_mapping - таблица перекодировки

In [44]:

class_mapping = {label:idx for idx,label in enumerate(numpy.unique(df_03['classlabel'])) }

class_mapping


{" 'class1'": 0, " 'class2'": 1}

In [1]:
#  Отступление о функции enumerate.
#  Генерирует кортежи, состоящие из двух элементов - индекса элемента и самого элемента.

#  Пример 1
a = [10, 20, 30, 40]
for i in enumerate(a):
     print(i)


(0, 10)
(1, 20)
(2, 30)
(3, 40)


In [25]:
#  Пример 2

b = "hello"
for i in enumerate(b):
     print(i)



(0, 'h')
(1, 'e')
(2, 'l')
(3, 'l')
(4, 'o')


In [ ]:
доделаем перекодировку

In [45]:
#  size_mapping = { " XL": 3, " L": 2, " M": 1}

df_03['classlabel_2'] = df_03['classlabel'].map(class_mapping)

print(df_03)


     color   size  price classlabel  size_2 size_3  classlabel_2
0  'green'    'M'   10.1   'class1'       1      M             0
1    'red'    'L'   13.5   'class2'       2      L             1
2   'blue'   'XL'   15.3   'class1'       3     XL             0


Другой способ сделать то же самое

In [ ]:
LabelEncoder класс в scikit-learn

In [48]:
from sklearn.preprocessing import LabelEncoder

class_le = LabelEncoder()
y = class_le.fit_transform(df_03['classlabel'].values)
y


#  Комментарий. Можно в два шага, сначала fit, затем transform

array([0, 1, 0], dtype=int64)

In [ ]:
#  Обратное преобразование

class_le.inverse_transform(y)
array(['class1', 'class2', 'class1'], dtype=object)


In [ ]:
class_le = LabelEncoder()
y = class_le.fit_transform(df_03['size'].values)
y


one-hot encoding для переменных в номинальной шкале
Создаются бинарные переменные для каждого значения переменной

In [54]:
X = df_03[['color', 'size', 'price']].values
X

array([["'green'", " 'M'", 10.1],
       ["'red'", " 'L'", 13.5],
       ["'blue'", " 'XL'", 15.3]], dtype=object)

In [55]:
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, " 'M'", 10.1],
       [2, " 'L'", 13.5],
       [0, " 'XL'", 15.3]], dtype=object)

In [56]:
size_le = LabelEncoder()
X[:, 1] = size_le.fit_transform(X[:, 1])
X


#  size закодирован неудачно! Переменная в порядковой шкале, процедура не угадала.

array([[1, 1, 10.1],
       [2, 0, 13.5],
       [0, 2, 15.3]], dtype=object)

Почему плохо использовать переменную с таблицей перекодировки
blue = 0
green = 1
red = 2

Получается, что blue более похож на green, чем на red


In [57]:
X

array([[1, 1, 10.1],
       [2, 0, 13.5],
       [0, 2, 15.3]], dtype=object)

In [59]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features=[0])
X_1 = ohe.fit_transform(X).toarray()

print(X_1)

[[ 0.   1.   0.   1.  10.1]
 [ 0.   0.   1.   0.  13.5]
 [ 1.   0.   0.   2.  15.3]]


Важно. 

1 Сначала LabelEncoder перекодирует в числа,
затем OneHotEncoder создает бинарные переменные.

2
Создание имен для новых столбцов - Ваша проблема!
OneHotEncoder имена не волнуют.
Но есть процедура, которая создает бинарные переменные и их имена